In [35]:
import MetaTrader5 as mt
import matplotlib.pyplot as plt 
plt.style.use("seaborn")
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
import time
from scipy.optimize import brute
import numpy as np
import ta

In [36]:
import talib

In [37]:
from Broker import Broker 

In [38]:
xm = Broker("xm", mt, demo = True)

Initialized


In [39]:
def getDataDF(ticker,time_frame,broker,time_steps=10000):
    broker.setTimeFrame(time_frame)
    broker.setTick(ticker)
    return broker.getPastData(time_steps=time_steps)

In [40]:
tickers = ["ETHUSD#,BTCUSD#","AUDUSD#","AUDJPY#","AUDNZD#","AUDCHF#","EURUSD#","EURJPY#","EURAUD#","EURNZD#","EURCHF#","US100Cash","GOLD#","GBPUSD#","GBPJPY#","GBPAUD#","GBPCHF#","GBPNZD#","USDCAD#","GBPCAD#","EURCAD#"]
time_frames = ["M5","M10","M15","M20","M30","H1","H2","H3","H4","D"]

In [57]:
data = getDataDF("EURUSD#","15M",xm,time_steps=2000)#.close.to_frame()

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume'], dtype='object')


In [58]:
data.to_hdf("XMGLobal-demo.h5",key="GOLD#")

c:\users\india\anaconda3\envs\dl\lib\site-packages\tables\path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'GOLD#'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though



In [59]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 1000)


def Ichimokubacktest(df: pd.DataFrame, tenkan_period: int, kijun_period: int):

    # Tenkan Sen : Short-term signal line

    df["rolling_min_tenkan"] = df["low"].rolling(window=tenkan_period).min()
    df["rolling_max_tenkan"] = df["high"].rolling(window=tenkan_period).max()

    df["tenkan_sen"] = (df["rolling_max_tenkan"] + df["rolling_min_tenkan"]) / 2

    df.drop(["rolling_min_tenkan", "rolling_max_tenkan"], axis=1, inplace=True)

    # Kijun Sen : Long-term signal line

    df["rolling_min_kijun"] = df["low"].rolling(window=kijun_period).min()
    df["rolling_max_kijun"] = df["high"].rolling(window=kijun_period).max()

    df["kijun_sen"] = (df["rolling_max_kijun"] + df["rolling_min_kijun"]) / 2

    df.drop(["rolling_min_kijun", "rolling_max_kijun"], axis=1, inplace=True)

    # Senkou Span A

    df["senkou_span_a"] = ((df["tenkan_sen"] + df["kijun_sen"]) / 2).shift(kijun_period)

    # Senkou Span B

    df["rolling_min_senkou"] = df["low"].rolling(window=kijun_period * 2).min()
    df["rolling_max_senkou"] = df["high"].rolling(window=kijun_period * 2).max()

    df["senkou_span_b"] = ((df["rolling_max_senkou"] + df["rolling_min_senkou"]) / 2).shift(kijun_period)

    df.drop(["rolling_min_senkou", "rolling_max_senkou"], axis=1, inplace=True)

    # Chikou Span : Confirmation line

    df["chikou_span"] = df["close"].shift(kijun_period)

    df.dropna(inplace=True)

    # Signal

    df["tenkan_minus_kijun"] = df["tenkan_sen"] - df["kijun_sen"]
    df["prev_tenkan_minus_kijun"] = df["tenkan_minus_kijun"].shift(1)

    df["signal"] = np.where((df["tenkan_minus_kijun"] > 0) &
                            (df["prev_tenkan_minus_kijun"] < 0) &
                            (df["close"] > df["senkou_span_a"]) &
                            (df["close"] > df["senkou_span_b"]) &
                            (df["close"] > df["chikou_span"]), 1,

                            np.where((df["tenkan_minus_kijun"] < 0) &
                            (df["prev_tenkan_minus_kijun"] > 0) &
                            (df["close"] < df["senkou_span_a"]) &
                            (df["close"] < df["senkou_span_b"]) &
                            (df["close"] < df["chikou_span"]), -1, 0))

    signal_data = df[df["signal"] != 0].copy()

    signal_data["pnl"] = signal_data["close"].pct_change() * signal_data["signal"].shift(1)

    return signal_data["pnl"].sum()

In [60]:
Ichimokubacktest(data,9,26)

0.04334188186900323